In [1]:
# -*- coding: cp1252 -*-


from mechanize import Browser
import os
import pandas as pd
import html5lib
from bs4 import BeautifulSoup  


#os.chdir(r"E:\Dropbox\Dropbox\0_DB\A_Elaborazioni\Coeweb")

class coeweb(object):
    
    def __init__(self, link):
        self.pandas_db = pd.DataFrame()
        self.link = link
        br = Browser()
        br.open(self.link)
        br.select_form(nr=0)
        
        self.paesi = []
        self.paesi_descizione = []
        control = br.form.find_control("PAESE")
        if control.type == "select":  
            for item in control.items:
                self.paesi.append(item.name)
                self.paesi_descizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.territori = []
        self.territori_descrizione = []
        control = br.form.find_control("CODTERRITORIO")
        if control.type == "select":  
            for item in control.items:
                self.territori.append((item.name, str([label.text  for label in item.get_labels()])))
                self.territori_descrizione.append((item.name, str([label.text  for label in item.get_labels()])))
                
        self.anni = []
        control = br.form.find_control("ANNO")
        if control.type == "select":  
            for item in control.items:
                self.anni.append((item.name, str([label.text  for label in item.get_labels()])))
        self.mese = []
        control = br.form.find_control("MESE")
        if control.type == "select": 
            for item in control.items:
                self.mese.append((item.name, str([label.text  for label in item.get_labels()])))
        self.cumulato = []
        control = br.form.find_control("TIPOVALORE")
        for item in control.items:
            self.cumulato.append((item.name, str([label.text  for label in item.get_labels()])))
        
        self.bric = ['508', '720', '664', '75']
        #[('508', "['508 - Brasile']"), ('720', "['720 - Cina']"), ('664', "['664 - India']"), ('75', "['75 - Russia']")]
    
    def convert_string_euro(self,x):
        '''questa funzione rimuove la formattazione dei numeri di coeweb, trasformando il testo in numeri'''
        try: 
            return float(x.replace(".", ""))
        except:
            pass
    

    def scarica_uno(self, territorio = ["309048"], paese = ["1033"], cumulato = ['M'], anno = ["2013"], mese = ["4"]  ):# , paese, anno, trimestre, modalita)
        
        br = Browser()
        br.open(self.link)

        br.select_form(nr=0)

        control_territorio = br.form.find_control("CODTERRITORIO")
        br[control_territorio.name] = territorio #["309048"]

        control_paese = br.form.find_control("PAESE")
        br[control_paese.name] = paese #["309048"]

        control_anno = br.form.find_control("ANNO")
        br[control_anno.name] = anno #["309048"]

        control_mese = br.form.find_control("MESE")
        br[control_mese.name] = mese #["309048"]

        control_cumulato = br.form.find_control("TIPOVALORE")
        br[control_cumulato.name] = cumulato #["309048"]

        nome_stringa = territorio[0] + "_" + paese[0] + "_" + anno[0] + "_" + mese[0] + "_" + cumulato[0]


        response = br.submit()
        out_file = open(nome_stringa +".html", "w")
        out_file.write(response.read()) 
        out_file.close()
        #os.system("start " + nome_stringa +".html")
        self.dati = pd.read_html(nome_stringa +".html", header= 0, index_col=0, encoding = "cp1252")#'utf8'
        #os.remove(territorio[0] +".html")
        print "download_" + nome_stringa
        br.close()
        return self.dati
        
        # i dati dovrebbero essere raccolti in un pandas 
        
    def scarica_molti(self, territorio = ["309048"],   lista_paesi = None, lista_anni = None, lista_mesi = None, lista_cumulato = ["M"]):
        ''' questa funzione permette di scaricare per uno o più territori uno o più settori, per uno o più periodi di tempo'''
        #self.pandas_db = pd.DataFrame()
        
        def nome_paese(dati, codice):
            for item in dati:
                if item[0] == codice:
                    pulito1= item[1].split("'",)[1]
                    pulito2 = pulito1.split(" - ",)[1]
                    print pulito2
                    return pulito2
        
        def nome_territorio(dati, codice):
            for item in dati:
                if item[0] == codice:
                    try:
                        pulito1= item[1].split("'")[1]
                        pulito1 = pulito1.replace("-","")
                        pulito1 = pulito1.replace("'","")
                        return pulito1
                    except:
                        pass
        
        
        for item_territori in territorio:
            for item_paesi in lista_paesi:
                for item_anni in lista_anni:
                    for item_mesi in lista_mesi:

                        dataset =  self.scarica_uno([item_territori], paese = [str(item_paesi)],  anno = [str(item_anni)], mese = [str(item_mesi)], cumulato = lista_cumulato)
                        dataset = dataset[1]#.set_index(territorio, str(item_paesi), str(item_anni), str(item_mesi)])
                        
                        
                        dataset["territorio"] = nome_territorio(self.territori_descrizione,  item_territori)  
                        #dataset["territorio"] = item_territori
                        
                        #nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["paese"] = nome_paese(self.paesi_descizione, str(item_paesi))
                        dataset["anno"] = str(item_anni)
                        dataset["mese"] = str(item_mesi)
                        dataset["cumulato"] = lista_cumulato[0]
                        dataset.set_index(["territorio","paese","anno","mese","cumulato"], append = True, inplace = True)
                        dataset["DATI EXPORT"] = dataset["EXP" + str(item_anni)]
                          
                        
                        
                        dataset = dataset["DATI EXPORT"].apply(lambda x: self.convert_string_euro(x))
                        
                        
                        
                        # datasetcompleto
                        self.pandas_db = pd.concat([self.pandas_db, pd.DataFrame(dataset)])
                        # settori ateco disponibili
                        self.pandas_db.settori = set(self.pandas_db.reset_index()["Divisioni"])
                        
                            #print "errore_" + territorio[0] + "_" + str(item_paesi) + "_" + str(item_anni) + "_" + str(item_mesi) + "_" + lista_cumulato[0]



        print "eseguito"
    
    
    
    def analizza_settori(self, settori, interpolazione = "no"):
        ''' questa funzione restituisce un dataframe e un dizionario contenente le serie storiche per i settori inseriti come argomento [lista]. 
        Effettua anche l'interpolazione dei dati mancanti'''
        self.dict_settore = {}
        for item in settori:
            dati_settore = self.pandas_db.xs(item, level=0)
            if interpolazione == "yes":
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
            else:
                dati_settore["DATI EXPORT"] = dati_settore["DATI EXPORT"].apply(lambda x: convert_string_euro(x))
            self.dict_settore[item] = dati_settore["DATI EXPORT"]
        self.dataframe_settori = pd.DataFrame(self.dict_settore)
        return self.dataframe_settori

In [2]:
def convert_string_euro(x):
    try: 
        return float(x.replace(".", ""))
    except:
        pass

In [3]:
os.getcwd()

'/home/scarselli/Scrivania/Coeweb Export/Coeweb Export'

In [4]:

lista_anni = [str(i) for i in range(2009,2016)]


In [5]:
Firenze = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [6]:
Firenze.paesi_descizione

[('1', "['1 - Francia']"),
 ('2', "['2 - Belgio e Lussemburgo']"),
 ('3', "['3 - Paesi Bassi']"),
 ('4', "['4 - Germania']"),
 ('6', "['6 - Regno Unito']"),
 ('7', "['7 - Irlanda']"),
 ('8', "['8 - Danimarca']"),
 ('9', "['9 - Grecia']"),
 ('10', "['10 - Portogallo']"),
 ('11', "['11 - Spagna']"),
 ('17', "['17 - Belgio']"),
 ('18', "['18 - Lussemburgo']"),
 ('21', "['21 - Ceuta']"),
 ('22', "['22 - Ceuta e Melilla']"),
 ('23', "['23 - Melilla']"),
 ('24', "['24 - Islanda']"),
 ('25', "['25 - Isole F\\xe6r\\xf8er']"),
 ('27', "['27 - Svalbard (arcipelago delle)']"),
 ('28', "['28 - Norvegia']"),
 ('30', "['30 - Svezia']"),
 ('32', "['32 - Finlandia']"),
 ('37', "['37 - Liechtenstein']"),
 ('38', "['38 - Austria']"),
 ('39', "['39 - Svizzera']"),
 ('41', "['41 - Faer \\xd8er']"),
 ('43', "['43 - Andorra']"),
 ('44', "['44 - Gibilterra']"),
 ('45', "['45 - Santa Sede (Stato della Citt\\xe0 del Vaticano)']"),
 ('46', "['46 - Malta']"),
 ('47', "['47 - San Marino']"),
 ('48', "['48 - Iugos

In [7]:
Firenze.territori_descrizione

toscana = [('309', "['---Toscana']"),
 ('309045', "['------Massa-Carrara']"),
 ('309046', "['------Lucca']"),
 ('309047', "['------Pistoia']"),
 ('309048', "['------Firenze']"),
 ('309049', "['------Livorno']"),
 ('309050', "['------Pisa']"),
 ('309051', "['------Arezzo']"),
 ('309052', "['------Siena']"),
 ('309053', "['------Grosseto']"),
 ('309100', "['------Prato']")]

In [8]:
metropolitana = [('309', "['---Toscana']"),
 ('309047', "['------Pistoia']"),
 ('309048', "['------Firenze']"),
 ('309100', "['------Prato']")]

In [9]:
CodiciToscana = []
for item in toscana:
    CodiciToscana.append(item[0])

In [10]:
CodiciMetropolitana = []
for item in metropolitana:
    CodiciMetropolitana.append(item[0])

In [11]:
Firenze.scarica_molti(territorio = CodiciMetropolitana, lista_paesi = ['6'], lista_anni = lista_anni, lista_mesi = ["9"], lista_cumulato = ['C'])

download_309_6_2009_9_C
Regno Unito
download_309_6_2010_9_C
Regno Unito
download_309_6_2011_9_C
Regno Unito
download_309_6_2012_9_C
Regno Unito
download_309_6_2013_9_C
Regno Unito
download_309_6_2014_9_C
Regno Unito
download_309_6_2015_9_C
Regno Unito
download_309047_6_2009_9_C
Regno Unito
download_309047_6_2010_9_C
Regno Unito
download_309047_6_2011_9_C
Regno Unito
download_309047_6_2012_9_C
Regno Unito
download_309047_6_2013_9_C
Regno Unito
download_309047_6_2014_9_C
Regno Unito
download_309047_6_2015_9_C
Regno Unito
download_309048_6_2009_9_C
Regno Unito
download_309048_6_2010_9_C
Regno Unito
download_309048_6_2011_9_C
Regno Unito
download_309048_6_2012_9_C
Regno Unito
download_309048_6_2013_9_C
Regno Unito
download_309048_6_2014_9_C
Regno Unito
download_309048_6_2015_9_C
Regno Unito
download_309100_6_2009_9_C
Regno Unito
download_309100_6_2010_9_C
Regno Unito
download_309100_6_2011_9_C
Regno Unito
download_309100_6_2012_9_C
Regno Unito
download_309100_6_2013_9_C
Regno Unito
downloa

In [12]:
dati_toscana_completi = Firenze.pandas_db

In [13]:
dati_toscana_completi.to_excel("dati_metropolitana_completi.xlsx")

In [18]:
Firenze

In [19]:
dati_toscana_completi[dati_toscana_completi.index.get_level_values('Divisioni').isin(['Totale'])]

DATI EXPORT
Divisioni territorio paese       anno mese cumulato             
Totale    Toscana    Regno Unito 2009 9    C          1528747209
                                 2010 9    C          1473169117
                                 2011 9    C          1519612834
                                 2012 9    C          1486383367
                                 2013 9    C          1615315910
                                 2014 9    C          1686906845
                                 2015 9    C          1802210027
          Pistoia    Regno Unito 2009 9    C            85316457
                                 2010 9    C            99959658
                                 2011 9    C           104536115
                                 2012 9    C            97534907
                                 2013 9    C           105228732
                                 2014 9    C           110815722
                                 2015 9    C           124232951
          Firenze    Regno Unito 2009 9    C           290155775
                                 2010 9    C           362372643
                                 2011 9    C           335666990
                                 2012 9    C           361374932
                                 2013 9    C           512352240
                                 2014 9    C           535248934
                                 2015 9    C           581639899
          Prato      Regno Unito 2009 9    C            82130808
                                 2010 9    C           114396829
                                 2011 9    C           108705991
                                 2012 9    C            90601437
                                 2013 9    C            93673380
                                 2014 9    C           111508702
                                 2015 9    C           142706401

In [20]:
Firenze_bric.to_excel("firenze_bric_db.xlsx") 

NameError: name 'Firenze_bric' is not defined

In [125]:
#dir(Firenze)
def nome_territorio(dati, codice):
    for item in dati:
        if item[0] == codice:
            try:
                pulito1= item[1].split("'")[1]
                pulito1 = pulito1.replace("-","")
                pulito1 = pulito1.replace("'","")
                return pulito1
            except:
                pass

In [126]:
        
nome_territorio(Firenze.territori_descrizione, "309048")

'Firenze'

In [67]:
settori_da_analizzare = [
    "CA10-Prodotti alimentari",
    "CA11-Bevande",
    "CB13-Prodotti tessili",
    "CB14-Articoli di abbigliamento (anche in pelle e in pelliccia)", 
    "CB15-Articoli in pelle (escluso abbigliamento) e simili",
]

In [68]:
settori_da_analizzare = list(Firenze.pandas_db.settori)

In [69]:
settori_da_analizzare

[nan,
 u'MC74-Prodotti delle altre attivit\xe0 professionali, scientifiche e tecniche',
 u"AA03-Prodotti della pesca e dell'acquacoltura",
 u'CM32-Prodotti delle altre industrie manifatturiere',
 u'CD19-Coke e prodotti derivanti dalla raffinazione del petrolio',
 u'Totale',
 u'CK28-Macchinari e apparecchiature nca',
 u'BB08-Altri minerali da cave e miniere',
 u'CA11-Bevande',
 u'CL30-Altri mezzi di trasporto',
 u'CB13-Prodotti tessili',
 u'CA10-Prodotti alimentari',
 u'CA12-Tabacco',
 u'CF21-Prodotti farmaceutici di base e preparati farmaceutici',
 u'JA59-Prodotti delle attivit\xe0 di produzione cinematografica, video e programmi televisivi; registrazioni musicali e sonore',
 u'CG22-Articoli in gomma e materie plastiche',
 u'RR91-Prodotti delle attivit\xe0 di biblioteche, archivi, musei e di altre attivit\xe0 culturali',
 u'AA01-Prodotti agricoli, animali e della caccia',
 u'CG23-Altri prodotti della lavorazione di minerali non metalliferi',
 u'CH24-Prodotti della metallurgia',
 u"EE38

In [70]:
from numpy import nan

In [71]:
cleanedList

[nan,
 u'MC74-Prodotti delle altre attivit\xe0 professionali, scientifiche e tecniche',
 u"AA03-Prodotti della pesca e dell'acquacoltura",
 u'CM32-Prodotti delle altre industrie manifatturiere',
 u'CD19-Coke e prodotti derivanti dalla raffinazione del petrolio',
 u'Totale',
 u'CK28-Macchinari e apparecchiature nca',
 u'BB08-Altri minerali da cave e miniere',
 u'CA11-Bevande',
 u'CL30-Altri mezzi di trasporto',
 u'CB13-Prodotti tessili',
 u'CA10-Prodotti alimentari',
 u'CA12-Tabacco',
 u'CF21-Prodotti farmaceutici di base e preparati farmaceutici',
 u'JA59-Prodotti delle attivit\xe0 di produzione cinematografica, video e programmi televisivi; registrazioni musicali e sonore',
 u'CG22-Articoli in gomma e materie plastiche',
 u'RR91-Prodotti delle attivit\xe0 di biblioteche, archivi, musei e di altre attivit\xe0 culturali',
 u'AA01-Prodotti agricoli, animali e della caccia',
 u'CG23-Altri prodotti della lavorazione di minerali non metalliferi',
 u'CH24-Prodotti della metallurgia',
 u"EE38

In [72]:
cleanedList[1:]

[u'MC74-Prodotti delle altre attivit\xe0 professionali, scientifiche e tecniche',
 u"AA03-Prodotti della pesca e dell'acquacoltura",
 u'CM32-Prodotti delle altre industrie manifatturiere',
 u'CD19-Coke e prodotti derivanti dalla raffinazione del petrolio',
 u'Totale',
 u'CK28-Macchinari e apparecchiature nca',
 u'BB08-Altri minerali da cave e miniere',
 u'CA11-Bevande',
 u'CL30-Altri mezzi di trasporto',
 u'CB13-Prodotti tessili',
 u'CA10-Prodotti alimentari',
 u'CA12-Tabacco',
 u'CF21-Prodotti farmaceutici di base e preparati farmaceutici',
 u'JA59-Prodotti delle attivit\xe0 di produzione cinematografica, video e programmi televisivi; registrazioni musicali e sonore',
 u'CG22-Articoli in gomma e materie plastiche',
 u'RR91-Prodotti delle attivit\xe0 di biblioteche, archivi, musei e di altre attivit\xe0 culturali',
 u'AA01-Prodotti agricoli, animali e della caccia',
 u'CG23-Altri prodotti della lavorazione di minerali non metalliferi',
 u'CH24-Prodotti della metallurgia',
 u"EE38-Prodo

In [73]:
Firenze.analizza_settori(cleanedList[1:])

-c:135: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


AA01-Prodotti agricoli, animali e della caccia  \
territorio paese anno mese cumulato                                                   
309048     75    2000 9    C                                                      0   
                 2001 9    C                                                  13025   
                 2002 9    C                                                  15566   
                 2003 9    C                                                      0   
                 2004 9    C                                                 770260   
                 2005 9    C                                                2061656   
                 2006 9    C                                                    NaN   
                 2007 9    C                                                 311569   
                 2008 9    C                                                  66951   
                 2009 9    C                                                   9694   
                 2010 9    C                                                  29423   
                 2011 9    C                                                 127306   
                 2012 9    C                                                 646598   
                 2013 9    C                                                 896766   

                                     AA02-Prodotti della silvicoltura  \
territorio paese anno mese cumulato                                     
309048     75    2000 9    C                                        0   
                 2001 9    C                                        0   
                 2002 9    C                                        0   
                 2003 9    C                                        0   
                 2004 9    C                                        0   
                 2005 9    C                                      NaN   
                 2006 9    C                                      NaN   
                 2007 9    C                                      NaN   
                 2008 9    C                                      NaN   
                 2009 9    C                                      NaN   
                 2010 9    C                                      NaN   
                 2011 9    C                                      NaN   
                 2012 9    C                                      NaN   
                 2013 9    C                                      NaN   

                                     AA03-Prodotti della pesca e dell'acquacoltura  \
territorio paese anno mese cumulato                                                  
309048     75    2000 9    C                                                   NaN   
                 2001 9    C                                                   NaN   
                 2002 9    C                                                   NaN   
                 2003 9    C                                                   NaN   
                 2004 9    C                                                   NaN   
                 2005 9    C                                                   NaN   
                 2006 9    C                                                   NaN   
                 2007 9    C                                                   NaN   
                 2008 9    C                                                   NaN   
                 2009 9    C                                                   NaN   
                 2010 9    C                                                   NaN   
                 2011 9    C                                                   NaN   
                 2012 9    C                                                   NaN   
                 2013 9    C                                                  2383   

                                     BB07-Minerali metalliferi  \
territorio paese anno mese cumul

In [58]:
Firenze.analizza_settori(cleanedList[1:])

-c:132: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


AA01-Prodotti agricoli, animali e della caccia  \
territorio paese anno mese cumulato                                                   
309048     75    2000 9    C                                                    0.0   
                 2001 9    C                                                13025.0   
                 2002 9    C                                                15566.0   
                 2003 9    C                                                    0.0   
                 2004 9    C                                               770260.0   
                 2005 9    C                                              2061656.0   
                 2006 9    C                                              1186612.5   
                 2007 9    C                                               311569.0   
                 2008 9    C                                                66951.0   
                 2009 9    C                                                 9694.0   
                 2010 9    C                                                29423.0   
                 2011 9    C                                               127306.0   
                 2012 9    C                                               646598.0   
                 2013 9    C                                               896766.0   

                                     AA02-Prodotti della silvicoltura  \
territorio paese anno mese cumulato                                     
309048     75    2000 9    C                                        0   
                 2001 9    C                                        0   
                 2002 9    C                                        0   
                 2003 9    C                                        0   
                 2004 9    C                                        0   
                 2005 9    C                                      NaN   
                 2006 9    C                                      NaN   
                 2007 9    C                                      NaN   
                 2008 9    C                                      NaN   
                 2009 9    C                                      NaN   
                 2010 9    C                                      NaN   
                 2011 9    C                                      NaN   
                 2012 9    C                                      NaN   
                 2013 9    C                                      NaN   

                                     AA03-Prodotti della pesca e dell'acquacoltura  \
territorio paese anno mese cumulato                                                  
309048     75    2000 9    C                                                   NaN   
                 2001 9    C                                                   NaN   
                 2002 9    C                                                   NaN   
                 2003 9    C                                                   NaN   
                 2004 9    C                                                   NaN   
                 2005 9    C                                                   NaN   
                 2006 9    C                                                   NaN   
                 2007 9    C                                                   NaN   
                 2008 9    C                                                   NaN   
                 2009 9    C                                                   NaN   
                 2010 9    C                                                   NaN   
                 2011 9    C                                                   NaN   
                 2012 9    C                                                   NaN   
                 2013 9    C                                                  2383   

                                     BB07-Minerali metalliferi  \
territorio paese anno mese cumul

In [38]:
Firenze.dataframe_settori[0]

CB14-Articoli di abbigliamento (anche in pelle e in pelliccia)  \
territorio paese anno mese cumulato                                                                   
309048     75    2000 9    C                                                  12579922                
                 2001 9    C                                                  19739363                
                 2002 9    C                                                  24227762                
                 2003 9    C                                                  24978527                
                 2004 9    C                                                  27302141                
                 2005 9    C                                                  40369183                
                 2006 9    C                                                  55409185                
                 2007 9    C                                                  70449187                
                 2008 9    C                                                  83028888                
                 2009 9    C                                                  41561791                
                 2010 9    C                                                  27123716                
                 2011 9    C                                                  45113391                
                 2012 9    C                                                  47639290                
                 2013 9    C                                                  57797345                

                                     CB15-Articoli in pelle (escluso abbigliamento) e simili  
territorio paese anno mese cumulato                                                           
309048     75    2000 9    C                                                 3074113.0        
                 2001 9    C                                                 5937865.0        
                 2002 9    C                                                 4469771.0        
                 2003 9    C                                                 5557355.0        
                 2004 9    C                                                 7071724.0        
                 2005 9    C                                                10479503.0        
                 2006 9    C                                                17372926.5        
                 2007 9    C                                                24266350.0        
                 2008 9    C                                                34813104.0        
                 2009 9    C                                                21669093.0        
                 2010 9    C                                                20307464.0        
                 2011 9    C                                                24938104.0        
                 2012 9    C                                                31612137.0        
                 2013 9    C                                                51948468.0

CB14-Articoli di abbigliamento (anche in pelle e in pelliccia)  \
territorio paese anno mese cumulato                                                                   
309048     75    2000 9    C                                                  12579922                
                 2001 9    C                                                  19739363                
                 2002 9    C                                                  24227762                
                 2003 9    C                                                  24978527                
                 2004 9    C                                                  27302141                
                 2005 9    C                                                  40369183                
                 2006 9    C                                                  55409185                
                 2007 9    C                                                  70449187                
                 2008 9    C                                                  83028888                
                 2009 9    C                                                  41561791                
                 2010 9    C                                                  27123716                
                 2011 9    C                                                  45113391                
                 2012 9    C                                                  47639290                
                 2013 9    C                                                  57797345                

                                     CB15-Articoli in pelle (escluso abbigliamento) e simili  
territorio paese anno mese cumulato                                                           
309048     75    2000 9    C                                                 3074113.0        
                 2001 9    C                                                 5937865.0        
                 2002 9    C                                                 4469771.0        
                 2003 9    C                                                 5557355.0        
                 2004 9    C                                                 7071724.0        
                 2005 9    C                                                10479503.0        
                 2006 9    C                                                17372926.5        
                 2007 9    C                                                24266350.0        
                 2008 9    C                                                34813104.0        
                 2009 9    C                                                21669093.0        
                 2010 9    C                                                20307464.0        
                 2011 9    C                                                24938104.0        
                 2012 9    C                                                31612137.0        
                 2013 9    C                                                51948468.0

In [16]:
russia_pelle = Firenze.pandas_db.xs(settori_da_analizzare, level=0)

KeyError: 'Key length (2) was greater than MultiIndex lexsort depth (0)'

In [73]:
russia_pelle["DATI EXPORT"] = russia_pelle["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()

-c:1: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


In [74]:
russia_abbigliamento = Firenze.pandas_db.xs("CB14-Articoli di abbigliamento (anche in pelle e in pelliccia)", level=0)

In [75]:
russia_abbigliamento["DATI EXPORT"] = russia_abbigliamento["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()

In [76]:
russia_tessile = Firenze.pandas_db.xs("CB13-Prodotti tessili", level=0)

In [77]:
russia_tessile["DATI EXPORT"] = russia_tessile["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()

In [78]:
russia_tessile

DATI EXPORT
territorio paese anno mese cumulato             
309048     75    2000 9    C             2770822
                 2001 9    C             4889621
                 2002 9    C             7549762
                 2003 9    C             6234286
                 2004 9    C             5683685
                 2005 9    C             3843475
                 2006 9    C             3618772
                 2007 9    C             3394069
                 2008 9    C             4469893
                 2009 9    C             1961118
                 2010 9    C             2637437
                 2011 9    C             3980858
                 2012 9    C             3978612
                 2013 9    C             4150549

In [79]:
russia_moda = russia_tessile + russia_abbigliamento + russia_pelle

In [80]:
russia_moda

DATI EXPORT
territorio paese anno mese cumulato             
309048     75    2000 9    C          18424857.0
                 2001 9    C          30566849.0
                 2002 9    C          36247295.0
                 2003 9    C          36770168.0
                 2004 9    C          40057550.0
                 2005 9    C          54692161.0
                 2006 9    C          76400883.5
                 2007 9    C          98109606.0
                 2008 9    C         122311885.0
                 2009 9    C          65192002.0
                 2010 9    C          50068617.0
                 2011 9    C          74032353.0
                 2012 9    C          83230039.0
                 2013 9    C         113896362.0

In [82]:
writer = pd.ExcelWriter('output.xlsx')
russia_pelle.to_excel(writer,'pelle')
russia_abbigliamento.to_excel(writer,'abbigliamento')
russia_tessile.to_excel(writer,'tessile')
russia_moda.to_excel(writer,'moda')
writer.save()

In [52]:
russia["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()

territorio  paese  anno  mese  cumulato
309048      75     2000  9     C            53714988.0
                   2001  9     C           100558311.0
                   2002  9     C           108431524.0
                   2003  9     C            82237656.0
                   2004  9     C           103713482.0
                   2005  9     C           305372412.0
                   2006  9     C           282533038.5
                   2007  9     C           259693665.0
                   2008  9     C           235949204.0
                   2009  9     C           178188726.0
                   2010  9     C           159871451.0
                   2011  9     C           234999174.0
                   2012  9     C           241138686.0
                   2013  9     C           299414649.0
Name: DATI EXPORT, dtype: float64

In [54]:
russia["DATI EXPORT"]

territorio  paese  anno  mese  cumulato
309048      75     2000  9     C            53.714.988
                   2001  9     C           100.558.311
                   2002  9     C           108.431.524
                   2003  9     C            82.237.656
                   2004  9     C           103.713.482
                   2005  9     C           305.372.412
                   2006  9     C                   NaN
                   2007  9     C           259.693.665
                   2008  9     C           235.949.204
                   2009  9     C           178.188.726
                   2010  9     C           159.871.451
                   2011  9     C           234.999.174
                   2012  9     C           241.138.686
                   2013  9     C           299.414.649
Name: DATI EXPORT, dtype: object

In [55]:
Firenze_russia = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [56]:
Firenze_russia.scarica_molti(lista_paesi = ['75'], lista_anni = lista_anni, lista_mesi = ["1","2","3","4"], lista_cumulato = ['M'])

download_309048_75_2000_1_M
download_309048_75_2000_2_M
download_309048_75_2000_3_M
download_309048_75_2000_4_M
download_309048_75_2001_1_M
download_309048_75_2001_2_M
download_309048_75_2001_3_M
download_309048_75_2001_4_M
download_309048_75_2002_1_M
download_309048_75_2002_2_M
download_309048_75_2002_3_M
download_309048_75_2002_4_M
download_309048_75_2003_1_M
download_309048_75_2003_2_M
download_309048_75_2003_3_M
download_309048_75_2003_4_M
download_309048_75_2004_1_M
download_309048_75_2004_2_M
download_309048_75_2004_3_M
download_309048_75_2004_4_M
download_309048_75_2005_1_M
download_309048_75_2005_2_M
download_309048_75_2005_3_M
download_309048_75_2005_4_M
download_309048_75_2006_1_M
download_309048_75_2006_2_M
download_309048_75_2006_3_M
download_309048_75_2006_4_M
download_309048_75_2007_1_M
download_309048_75_2007_2_M
download_309048_75_2007_3_M
download_309048_75_2007_4_M
download_309048_75_2008_1_M
download_309048_75_2008_2_M
download_309048_75_2008_3_M
download_309048_75_2

In [57]:
Firenze_russia.pandas_db.xs("Totale", level=0)

DATI EXPORT
territorio paese anno mese cumulato             
309048     75    2000 1    M          13.239.404
                      2    M          14.146.584
                      3    M          12.170.365
                      4    M          14.158.634
                 2001 1    M          12.085.556
                      2    M          17.768.444
                      3    M          17.541.660
                      4    M          53.162.651
                 2002 1    M          34.233.671
                      2    M          20.846.740
                      3    M          28.913.578
                      4    M          24.437.535
                 2003 1    M          17.565.604
                      2    M          16.285.126
                      3    M          21.136.771
                      4    M          27.250.155
                 2004 1    M          17.584.794
                      2    M          23.198.729
                      3    M          32.500.981
                      4    M          30.428.978
                 2005 1    M          45.869.480
                      2    M         108.329.489
                      3    M          90.091.169
                      4    M          61.082.274
                 2006 1    M          33.654.681
                      2    M          40.479.727
                      3    M          56.637.895
                      4    M          58.792.039
                 2007 1    M          47.271.757
                      2    M          45.375.475
                      3    M          96.276.024
                      4    M          70.770.409
                 2008 1    M          57.954.569
                      2    M          53.568.721
                      3    M          66.689.589
                      4    M          57.736.325
                 2009 1    M          38.885.134
                      2    M          59.668.296
                      3    M          43.450.336
                      4    M          36.184.960
                 2010 1    M          40.326.699
                      2    M          37.542.595
                      3    M          37.000.333
                      4    M          45.001.824
                 2011 1    M          51.633.825
                      2    M          43.052.194
                      3    M          68.336.192
                      4    M          71.976.963
                 2012 1    M          56.713.229
                      2    M          48.398.145
                      3    M          62.030.866
                      4    M          73.996.446
                 2013 1    M          86.718.677
                      2    M          54.905.672
                      3    M          98.179.667
                      4    M          59.610.633

In [ ]:

russia_pelle
russia_abbigliamento
russia_tessile
russia_moda


In [94]:
import matplotlib.pyplot as plt

In [95]:
x = lista_anni

In [96]:
x

['2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013']

In [99]:
plt.plot(x,russia_pelle["DATI EXPORT"])
plt.plot(x,russia_abbigliamento["DATI EXPORT"])
plt.plot(x,russia_tessile["DATI EXPORT"])
plt.plot(x,russia_moda["DATI EXPORT"])

In [100]:
plt.show()

In [23]:
Prato = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [24]:
Prato.territori

[('1', "['Italia Nord-occidentale']"),
 ('101', "['---Piemonte']"),
 ('101001', "['------Torino']"),
 ('101002', "['------Vercelli']"),
 ('101003', "['------Novara']"),
 ('101004', "['------Cuneo']"),
 ('101005', "['------Asti']"),
 ('101006', "['------Alessandria']"),
 ('101096', "['------Biella']"),
 ('101103', "['------Verbano-Cusio-Ossola']"),
 ('102', '["---Valle d\'Aosta/Vall\\xe9e d\'Aoste"]'),
 ('102007', "['------Aosta']"),
 ('103', "['---Lombardia']"),
 ('103012', "['------Varese']"),
 ('103013', "['------Como']"),
 ('103014', "['------Sondrio']"),
 ('103015', "['------Milano']"),
 ('103016', "['------Bergamo']"),
 ('103017', "['------Brescia']"),
 ('103018', "['------Pavia']"),
 ('103019', "['------Cremona']"),
 ('103020', "['------Mantova']"),
 ('103097', "['------Lecco']"),
 ('103098', "['------Lodi']"),
 ('103108', "['------Monza e della Brianza']"),
 ('107', "['---Liguria']"),
 ('107008', "['------Imperia']"),
 ('107009', "['------Savona']"),
 ('107010', "['------Genova'

In [21]:
#Prato.paesi_descizione

[('1', "['1 - Francia']"),
 ('2', "['2 - Belgio e Lussemburgo']"),
 ('3', "['3 - Paesi Bassi']"),
 ('4', "['4 - Germania']"),
 ('6', "['6 - Regno Unito']"),
 ('7', "['7 - Irlanda']"),
 ('8', "['8 - Danimarca']"),
 ('9', "['9 - Grecia']"),
 ('10', "['10 - Portogallo']"),
 ('11', "['11 - Spagna']"),
 ('17', "['17 - Belgio']"),
 ('18', "['18 - Lussemburgo']"),
 ('21', "['21 - Ceuta']"),
 ('22', "['22 - Ceuta e Melilla']"),
 ('23', "['23 - Melilla']"),
 ('24', "['24 - Islanda']"),
 ('25', "['25 - Isole F\\xe6r\\xf8er']"),
 ('27', "['27 - Svalbard (arcipelago delle)']"),
 ('28', "['28 - Norvegia']"),
 ('30', "['30 - Svezia']"),
 ('32', "['32 - Finlandia']"),
 ('37', "['37 - Liechtenstein']"),
 ('38', "['38 - Austria']"),
 ('39', "['39 - Svizzera']"),
 ('41', "['41 - Faer \\xd8er']"),
 ('43', "['43 - Andorra']"),
 ('44', "['44 - Gibilterra']"),
 ('45', "['45 - Santa Sede (Stato della Citt\\xe0 del Vaticano)']"),
 ('46', "['46 - Malta']"),
 ('47', "['47 - San Marino']"),
 ('48', "['48 - Iugos

In [26]:
Prato.scarica_molti(territorio = ['309100'], lista_paesi = ['75'], lista_anni = lista_anni, lista_mesi = ["9"], lista_cumulato = ['C'])

download_309100_75_2000_9_C
download_309100_75_2001_9_C
download_309100_75_2002_9_C
download_309100_75_2003_9_C
download_309100_75_2004_9_C
download_309100_75_2005_9_C
download_309100_75_2006_9_C
download_309100_75_2007_9_C
download_309100_75_2008_9_C
download_309100_75_2009_9_C
download_309100_75_2010_9_C
download_309100_75_2011_9_C
download_309100_75_2012_9_C
download_309100_75_2013_9_C
eseguito


In [27]:
Prato.pandas_db.xs("Totale", level=0)

DATI EXPORT
territorio paese anno mese cumulato            
309100     75    2000 9    C         41.164.567
                 2001 9    C         69.102.171
                 2002 9    C         65.398.909
                 2003 9    C         67.978.942
                 2004 9    C         63.422.877
                 2005 9    C         61.965.012
                 2006 9    C         81.485.512
                 2007 9    C         79.678.885
                 2008 9    C         81.923.831
                 2009 9    C         49.462.404
                 2010 9    C         61.070.919
                 2011 9    C         62.638.447
                 2012 9    C         62.142.354
                 2013 9    C         57.907.409

In [33]:
prato_russia_pelle =Prato.pandas_db.xs("CB15-Articoli in pelle (escluso abbigliamento) e simili", level=0)

In [37]:
prato_russia_pelle["DATI EXPORT"] = prato_russia_pelle["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()

-c:1: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


In [38]:
prato_russia_pelle

DATI EXPORT
territorio paese anno mese cumulato             
309100     75    2000 9    C              158632
                 2001 9    C              231660
                 2002 9    C              154799
                 2003 9    C              165903
                 2004 9    C              408330
                 2005 9    C              247950
                 2006 9    C              485980
                 2007 9    C              989799
                 2008 9    C             1533530
                 2009 9    C              583504
                 2010 9    C              508496
                 2011 9    C              335156
                 2012 9    C              236883
                 2013 9    C              281692

In [42]:
prato_russia_abbigliamento =Prato.pandas_db.xs("CB14-Articoli di abbigliamento (anche in pelle e in pelliccia)", level=0)
prato_russia_abbigliamento["DATI EXPORT"] = prato_russia_abbigliamento["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
prato_russia_abbigliamento

-c:2: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


DATI EXPORT
territorio paese anno mese cumulato             
309100     75    2000 9    C             1751976
                 2001 9    C             2834047
                 2002 9    C             2637805
                 2003 9    C             4272491
                 2004 9    C             7127862
                 2005 9    C             9249844
                 2006 9    C            14067698
                 2007 9    C            14605224
                 2008 9    C            16611356
                 2009 9    C             9289334
                 2010 9    C             6243681
                 2011 9    C             7201130
                 2012 9    C             7653014
                 2013 9    C             6779333

In [43]:
prato_russia_tessile =Prato.pandas_db.xs("CB13-Prodotti tessili", level=0)
prato_russia_tessile["DATI EXPORT"] = prato_russia_tessile["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
prato_russia_tessile

DATI EXPORT
territorio paese anno mese cumulato             
309100     75    2000 9    C            34583096
                 2001 9    C            59201427
                 2002 9    C            58877882
                 2003 9    C            57038863
                 2004 9    C            46241932
                 2005 9    C            46519498
                 2006 9    C            56888459
                 2007 9    C            49360000
                 2008 9    C            51626071
                 2009 9    C            34345469
                 2010 9    C            45965930
                 2011 9    C            44855837
                 2012 9    C            47925262
                 2013 9    C            42187800

In [45]:
prato_russia_moda = prato_russia_tessile + prato_russia_abbigliamento + prato_russia_pelle

In [46]:
prato_russia_moda 

DATI EXPORT
territorio paese anno mese cumulato             
309100     75    2000 9    C            36493704
                 2001 9    C            62267134
                 2002 9    C            61670486
                 2003 9    C            61477257
                 2004 9    C            53778124
                 2005 9    C            56017292
                 2006 9    C            71442137
                 2007 9    C            64955023
                 2008 9    C            69770957
                 2009 9    C            44218307
                 2010 9    C            52718107
                 2011 9    C            52392123
                 2012 9    C            55815159
                 2013 9    C            49248825

In [47]:
prato_russia_agricoli =Prato.pandas_db.xs("AA01-Prodotti agricoli, animali e della caccia", level=0)
prato_russia_agricoli["DATI EXPORT"] = prato_russia_agricoli["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
prato_russia_agricoli

DATI EXPORT
territorio paese anno mese cumulato             
309100     75    2000 9    C                   0
                 2001 9    C                   0
                 2002 9    C                   0
                 2003 9    C                3527
                 2004 9    C                 597
                 2005 9    C                5736
                 2006 9    C                   0
                 2007 9    C                1524
                 2008 9    C                   0
                 2009 9    C                   0
                 2010 9    C              415251
                 2011 9    C              301473
                 2012 9    C               58826
                 2013 9    C               20150

In [ ]:
prato_russia_tessile + prato_russia_abbigliamento + prato_russia_pelle + 
prato_russia_agricoli + prato_russia_moda 

In [48]:
prato_russia_Totale =Prato.pandas_db.xs("Totale", level=0)
prato_russia_Totale["DATI EXPORT"] = prato_russia_Totale["DATI EXPORT"].apply(lambda x: convert_string_euro(x)).interpolate()
prato_russia_Totale

DATI EXPORT
territorio paese anno mese cumulato             
309100     75    2000 9    C            41164567
                 2001 9    C            69102171
                 2002 9    C            65398909
                 2003 9    C            67978942
                 2004 9    C            63422877
                 2005 9    C            61965012
                 2006 9    C            81485512
                 2007 9    C            79678885
                 2008 9    C            81923831
                 2009 9    C            49462404
                 2010 9    C            61070919
                 2011 9    C            62638447
                 2012 9    C            62142354
                 2013 9    C            57907409

In [49]:
writer = pd.ExcelWriter('Prato.xlsx')
prato_russia_pelle.to_excel(writer,'pelle')
prato_russia_abbigliamento.to_excel(writer,'abbigliamento')
prato_russia_tessile.to_excel(writer,'tessile')
prato_russia_agricoli.to_excel(writer,'agricoli')
prato_russia_moda.to_excel(writer,'moda')
prato_russia_Totale.to_excel(writer,'Totale')
writer.save()

In [5]:
province = coeweb("http://www.coeweb.istat.it/predefinite/tutto_paese_merce.asp?livello=ATE07_AT2&riga=MERCE&territorio=S&AG=S")

In [29]:
toscanalist = []
toscanaspieg = []
for item in province.territori:
    if item[0][:3] =="309":
        toscanalist.append(item[0])
        toscanaspieg.append(item)

In [32]:
pd.DataFrame(toscanaspieg).to_excel("legenda.xlsx")

In [8]:
province.scarica_molti(territorio = [
 '309045',
 '309046',
 '309047',
 '309048',
 '309049'], lista_paesi = ["1033"], lista_anni = ["2009","2013"], lista_mesi = ["9"], lista_cumulato = ['C'])

download_309045_1033_2009_9_C
download_309045_1033_2013_9_C
download_309046_1033_2009_9_C
download_309046_1033_2013_9_C
download_309047_1033_2009_9_C
download_309047_1033_2013_9_C
download_309048_1033_2009_9_C
download_309048_1033_2013_9_C
download_309049_1033_2009_9_C
download_309049_1033_2013_9_C
eseguito


In [12]:
prima_trance = province.pandas_db

In [13]:
province.scarica_molti(territorio = [
 '309050',
 '309051',
 '309052',
 '309053',
 '309100'], lista_paesi = ["1033"], lista_anni = ["2009","2013"], lista_mesi = ["9"], lista_cumulato = ['C'])

download_309050_1033_2009_9_C
download_309050_1033_2013_9_C
download_309051_1033_2009_9_C
download_309051_1033_2013_9_C
download_309052_1033_2009_9_C
download_309052_1033_2013_9_C
download_309053_1033_2009_9_C
download_309053_1033_2013_9_C
download_309100_1033_2009_9_C
download_309100_1033_2013_9_C
eseguito


In [14]:
prima_trance.shape

(380, 1)

In [16]:
seconda_trance = province.pandas_db

In [18]:
completo = pd.concat([prima_trance, seconda_trance])

In [19]:
completo.Totale("Totale"

,,,,,,DATI EXPORT
Divisioni,territorio,paese,anno,mese,cumulato,
"AA01-Prodotti agricoli, animali e della caccia",309045,1033,2009,9,C,3.194.409
AA02-Prodotti della silvicoltura,309045,1033,2009,9,C,4.050
AA03-Prodotti della pesca e dell'acquacoltura,309045,1033,2009,9,C,0
BB07-Minerali metalliferi,309045,1033,2009,9,C,62.273
BB08-Altri minerali da cave e miniere,309045,1033,2009,9,C,111.182.088
CA10-Prodotti alimentari,309045,1033,2009,9,C,651.433
CA11-Bevande,309045,1033,2009,9,C,227.180
CA12-Tabacco,309045,1033,2009,9,C,390.091
CB13-Prodotti tessili,309045,1033,2009,9,C,2.033.377


In [20]:
completo

AttributeError: 'DataFrame' object has no attribute 'analizza_settori'

In [21]:
  def convert_string_euro(x):
        '''questa funzione rimuove la formattazione dei numeri di coeweb, trasformando il testo in numeri'''
        try: 
            return float(x.replace(".", ""))
        except:
            pass

In [25]:
completo = completo["DATI EXPORT"].apply(convert_string_euro)

In [28]:
pd.DataFrame(completo).to_excel("province_toscana.xlsx")